<a href="https://colab.research.google.com/github/SantosAbimaelRomero/Sales-Preditions/blob/main/Sales_Predictions_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Statistics - Part 1
- Santos Romero

## Loading Data

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
sales_csv = pd.read_csv('/content/drive/MyDrive/Stack Projects/Project 1, Sales Predictions/sales_predictions.csv')

## Data Cleaning

### Overview of Data

In [3]:
sales_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
sales_csv.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


General Info
- 8523 rows
- 11 Columns
> - Item_Weight and Outlet_Size have NaNs
> - No data type issues

### Checking for duplicates

In [5]:
sales_csv.duplicated().sum()
# Nothing was printed, contains no duplicates

0

### Missing Values

In [7]:
sales_csv.isna().sum()
# Item_Weight and Outlet_Size are missing data

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

#### Outlet_Size

Let's look for patterns with the other two related columns, Outlet_Location_Type and Outlet_Type

In [8]:
sales_csv.groupby('Outlet_Location_Type')['Outlet_Size'].value_counts()
# Tier 1s are mostly small but can be any size
# Tier 2s are all small
# Tier 3s are mostly medium but can be High

Outlet_Location_Type  Outlet_Size
Tier 1                Small          1458
                      Medium          930
Tier 2                Small           930
Tier 3                Medium         1863
                      High            932
Name: Outlet_Size, dtype: int64

In [9]:
sales_csv.groupby('Outlet_Type')['Outlet_Size'].value_counts()
# Grocery Stores are all small
# Supermarket Type1s are mostly small but can be any size
# Supermarket Type2s and Type3s are all medium

Outlet_Type        Outlet_Size
Grocery Store      Small           528
Supermarket Type1  Small          1860
                   High            932
                   Medium          930
Supermarket Type2  Medium          928
Supermarket Type3  Medium          935
Name: Outlet_Size, dtype: int64

Small
- Tier 2s are always small, grocery stores are always small
- Tier 1s are mostly small, Supermarket Type 1s are mostly small

Medium
- Tier 3s are mostly medium, Tier 1s are less likely but can be medium
- Supermarket Type 2 & 3 are only medium, Type 1s are less likely but can be medium

High
- Tier 3s are the only highs
- Supermarket Type 1s are the only highs

In [10]:
# Let's Start with what we are certain on
# Tier 2s are all small and Grocery Stores are all small
# Supermarket Type2s and Type3s are all medium
# Since the index for out Dataframe is just numbers from 0 - 8523
# Let's loop through the index
for index in range(len(sales_csv)):
    if sales_csv.loc[index, 'Outlet_Location_Type'] == 'Tier 2':
        sales_csv.loc[index, 'Outlet_Size'] = 'Small'
    elif sales_csv.loc[index, 'Outlet_Type'] == 'Grocery Store':
        sales_csv.loc[index, 'Outlet_Size'] = 'Small'
    elif sales_csv.loc[index, 'Outlet_Type'] == 'Supermarket Type2':
        sales_csv.loc[index, 'Outlet_Size'] = 'Medium'
    elif sales_csv.loc[index, 'Outlet_Type'] == 'Supermarket Type3':
        sales_csv.loc[index, 'Outlet_Size'] = 'Medium'

In [11]:
sales_csv.isna().sum()
# That seems to have been all of them

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

#### Item_Weight

For my purpose with this data, the weight of the items is irrelavent as I am not consered with shipping. Only the in person sales that occur at the supermarkets/grocery stores. I shall drop the column entirely.

In [12]:
# Drop Item_Weight Column
sales_csv.drop(columns='Item_Weight', inplace=True)
sales_csv.isna().sum()
# There are now no missing values in our data

Item_Identifier              0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

### Incorrect Values

In [13]:
sales_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                8523 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 732.6+ KB


Let's go through each column where this might be an issue

In [14]:
sales_csv['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [15]:
sales_csv['Item_Fat_Content'].replace('low fat', 'Low Fat', inplace=True)
sales_csv['Item_Fat_Content'].replace('LF', 'Low Fat', inplace=True)
sales_csv['Item_Fat_Content'].replace('reg', 'Regular', inplace=True)
sales_csv['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [16]:
sales_csv['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [17]:
sales_csv['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [18]:
sales_csv['Outlet_Size'].value_counts()

Small     4798
Medium    2793
High       932
Name: Outlet_Size, dtype: int64

In [19]:
sales_csv['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [20]:
sales_csv['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [21]:
sales_csv['Outlet_Establishment_Year'].value_counts()

1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64

Only Item_Fat_Content had value issues

## Exploratory Visuals

### Summary Statistics

In [22]:
# Basic Statistics for all relevant number columns
print(f"""Item Visibility:
Minimum:{sales_csv['Item_Visibility'].min()}
Maximum:{sales_csv['Item_Visibility'].max().round(4)}
Average:{sales_csv['Item_Visibility'].mean().round(4)}
""")
print(f"""Item MRP:
Minimum:${sales_csv['Item_MRP'].min()}
Maximum:${sales_csv['Item_MRP'].max().round(2)}
Average:${sales_csv['Item_MRP'].mean().round(2)}
""")
print(f"""Item Outlet Sales:
Minimum:${sales_csv['Item_Outlet_Sales'].min()}
Maximum:${sales_csv['Item_Outlet_Sales'].max().round(2)}
Average:${sales_csv['Item_Outlet_Sales'].mean().round(2)}
""")

Item Visibility:
Minimum:0.0
Maximum:0.3284
Average:0.0661

Item MRP:
Minimum:$31.29
Maximum:$266.89
Average:$140.99

Item Outlet Sales:
Minimum:$33.29
Maximum:$13086.96
Average:$2181.29

